# 引言

在数据预处理中，我一般是使用Python的Pandas将其整理成整洁的长表格，输入到其它语言中进行建模计算，然后也不是输入到其它语言就可以直接用的，还需要稍微做一点处理，比较典型的包含
1. 数据框读取与类型指定;
2. 变量选择（select）;
3. 生成新变量（mutate）;
4. 数据类型转换（将数据框转换为向量或矩阵）;
5. 分组聚合;
6. 操控变形（长变宽和宽变长）;
7. 变量分布图绘制。
其中（5， 6)是重点

由于Pandas已经做好了绝大部分的数据清理工作，DataFrame.jl我们也不需要学习的太过深入，掌握上述基本操作，懂一点DataFrame.jl的基本原理即可（这些数据分析库实现的功能大同小异，基本上都差不多，增删查改，操控变形，分组聚合等等）

参考资料：
+ 中文cheetsheet
+ https://github.com/bkamins
+ https://dataframes.juliadata.org/stable/man/comparisons/#Comparison-with-the-Python-package-pandas

# DataFrame.jl

In [36]:
# 设置显示的行数和列数
Base.displaysize() = (10, 800)

In [37]:
using DataFrames
using RDatasets
using Random
using Distributions

## DataFrame类型

创建可以通过`关键字参数`，`Pair`，`矩阵`，`表格`等建立

In [57]:
N = 5000
df = DataFrame(x1 = randn(N), x2 = randn(N), x3 = randn(N), intercept = 1)
df[!, :y] = Matrix(df) * [-1, 2, 3, 4] + randn(N)
using FixedEffectModels
reg(df, @formula(y ~ x1 + x2 + x3))

                               FixedEffectModel                               
Number of obs:                    5000  Converged:                        true
dof (model):                         3  dof (residuals):                  4995
R²:                              0.935  R² adjusted:                     0.935
F-statistic:              24129.989965  P-value:                         0.000
              Estimate  Std. Error      t-stat  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────────
x1           -1.002412    0.014165  -70.764397    <1e-99  -1.030183  -0.974642
x2            2.014420    0.013919  144.720924    <1e-99   1.987132   2.041708
x3            3.012791    0.014088  213.856320    <1e-99   2.985173   3.040410
(Intercept)   3.991898    0.014176  281.603083    <1e-99   3.964108   4.019689


In [58]:
N = 5000
df = DataFrame(x1 = randn(N), x2 = randn(N), x3 = randn(N), intercept = 1)
df[!, :y] = rand.(Poisson.(exp.(Matrix(df) * [-1, 2, -3, -4])))
using FixedEffectModels
reg(df, @formula(y ~ x1 + x2 + x3))

                                FixedEffectModel                                
Number of obs:                     5000  Converged:                         true
dof (model):                          3  dof (residuals):                   4995
R²:                               0.017  R² adjusted:                      0.016
F-statistic:                  28.394509  P-value:                          0.000
               Estimate  Std. Error     t-stat  Pr(>|t|)   Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────────
x1            -7.045645    3.185719  -2.211634    0.0270  -13.291053   -0.800238
x2            16.981155    3.172518   5.352580    <1e-07   10.761628   23.200682
x3           -23.288697    3.248803  -7.168392    <1e-12  -29.657778  -16.919616
(Intercept)    9.962896    3.199892   3.113510    0.0019    3.689704   16.236088


In [59]:
using GLM

In [60]:
glm(@formula(y ~ x1 + x2 + x3), df, Poisson(), LogLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Poisson{Float64}, LogLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

y ~ 1 + x1 + x2 + x3

Coefficients:
───────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error        z  Pr(>|z|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  -3.967890    0.022441  -176.82    <1e-99  -4.011873  -3.923907
x1           -0.992254    0.004010  -247.47    <1e-99  -1.000113  -0.984396
x2            1.998611    0.005866   340.74    <1e-99   1.987114   2.010107
x3           -2.990613    0.007255  -412.20    <1e-99  -3.004833  -2.976393
───────────────────────────────────────────────────────────────────────────

In [61]:
nrow(df), ncol(df)

(5000, 5)

In [62]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,x1,0.004719,-4.018530,0.002617,3.347035,0,Float64
2,x2,-0.008869,-3.972854,-0.003843,3.496787,0,Float64
3,x3,0.001581,-3.161158,-0.017333,3.750153,0,Float64
4,intercept,1.000000,1,1.000000,1,0,Int64
5,y,9.742200,0,0.000000,14088,0,Int64


In [63]:
# 获取列名
names(df)

5-element Vector{String}:
 "x1"
 "x2"
 "x3"
 "intercept"
 "y"

In [64]:
# 获取列名为 Symbol
propertynames(df)

5-element Vector{Symbol}:
 :x1
 :x2
 :x3
 :intercept
 :y

In [65]:
# 直接获取原数据（不会copy）
df.x1, df."x1", df.:x1, df[!, "x1"], df[!, :x1]

([-0.893089, -0.663048, -0.011093, 0.048355, 0.615507, 0.488753, -0.157378, -0.776902, -0.996300, 0.568948  …  0.086998, -1.425759, 0.575318, 0.166253, -0.377847, 0.794452, -0.618051, -0.270667, -3.007232, 1.452772], [-0.893089, -0.663048, -0.011093, 0.048355, 0.615507, 0.488753, -0.157378, -0.776902, -0.996300, 0.568948  …  0.086998, -1.425759, 0.575318, 0.166253, -0.377847, 0.794452, -0.618051, -0.270667, -3.007232, 1.452772], [-0.893089, -0.663048, -0.011093, 0.048355, 0.615507, 0.488753, -0.157378, -0.776902, -0.996300, 0.568948  …  0.086998, -1.425759, 0.575318, 0.166253, -0.377847, 0.794452, -0.618051, -0.270667, -3.007232, 1.452772], [-0.893089, -0.663048, -0.011093, 0.048355, 0.615507, 0.488753, -0.157378, -0.776902, -0.996300, 0.568948  …  0.086998, -1.425759, 0.575318, 0.166253, -0.377847, 0.794452, -0.618051, -0.270667, -3.007232, 1.452772], [-0.893089, -0.663048, -0.011093, 0.048355, 0.615507, 0.488753, -0.157378, -0.776902, -0.996300, 0.568948  …  0.086998, -1.425759, 0.57

In [66]:
# 复制原数据的一列
df[:, :x1], df[:, "x1"]

([-0.893089, -0.663048, -0.011093, 0.048355, 0.615507, 0.488753, -0.157378, -0.776902, -0.996300, 0.568948  …  0.086998, -1.425759, 0.575318, 0.166253, -0.377847, 0.794452, -0.618051, -0.270667, -3.007232, 1.452772], [-0.893089, -0.663048, -0.011093, 0.048355, 0.615507, 0.488753, -0.157378, -0.776902, -0.996300, 0.568948  …  0.086998, -1.425759, 0.575318, 0.166253, -0.377847, 0.794452, -0.618051, -0.270667, -3.007232, 1.452772])

## 文件读取

`CSV.read("/path/to/file", DataFrame)`，读取为DataFrame

+ integer, vecor of integers
+ Symbol, vector of Symbols
+ string, vector of strings
+ `All()`, `Between()`, `Not()`
+ 正则表达式

## 变量生成

In [69]:
select(df, :x1 => (a -> a * 1000) => :x1_mean)

,x1_mean
,Float64
1,-893.089008
2,-663.047503
3,-11.093019
4,48.355102
5,615.506890
6,488.753350
7,-157.378402
8,-776.902255
9,-996.299904


In [70]:
transform(df, :x1 => (a -> a * 1000) => :x1_mean)

,x1,x2,x3,intercept,y,x1_mean
,Float64,Float64,Float64,Int64,Int64,Float64
1,-0.893089,1.090354,-0.098414,1,1,-893.089008
2,-0.663048,-0.599173,1.071210,1,0,-663.047503
3,-0.011093,1.131863,0.619766,1,0,-11.093019
4,0.048355,-0.449926,0.392205,1,0,48.355102
5,0.615507,-1.815548,-0.266469,1,0,615.506890
6,0.488753,-1.866037,-1.411774,1,0,488.753350
7,-0.157378,-1.058923,0.220755,1,0,-157.378402
8,-0.776902,-0.906232,-0.521686,1,0,-776.902255
9,-0.996300,-0.084873,0.061557,1,0,-996.299904


## 分组聚合

三个函数：
+ `combine`,类似于`apply`，可以处理`df`，也可以处理`groupby(df)`,对于返回结果没有强制要求
+ `select`,类似于`transform`,可以处理`df`，也可以处理`groupby(df)`，返回结果和原DataFrame行索引一致，仅包含新生成的列
+ `transform`,类似于`transform`,可以处理`df`，也可以处理`groupby(df)`，返回结果和原DataFrame行索引一致，包含DataFrame原有的列和新生成的列

In [76]:
# 示例数据
iris = dataset("datasets", "iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.100000,3.500000,1.400000,0.200000,setosa
2,4.900000,3.000000,1.400000,0.200000,setosa
3,4.700000,3.200000,1.300000,0.200000,setosa
4,4.600000,3.100000,1.500000,0.200000,setosa
5,5.000000,3.600000,1.400000,0.200000,setosa
6,5.400000,3.900000,1.700000,0.400000,setosa
7,4.600000,3.400000,1.400000,0.300000,setosa
8,5.000000,3.400000,1.500000,0.200000,setosa
9,4.400000,2.900000,1.400000,0.200000,setosa


In [77]:
# 生成两列，这两列是对原来数据列的数值变换
combine(
    iris,
    :SepalLength => (c -> c * 100) => "SepalLength*100",
    :PetalLength => (c -> c / 100) => "SepalLength/100",
)

,SepalLength*100,SepalLength/100
,Float64,Float64
1,510.000000,0.014000
2,490.000000,0.014000
3,470.000000,0.013000
4,460.000000,0.015000
5,500.000000,0.014000
6,540.000000,0.017000
7,460.000000,0.014000
8,500.000000,0.015000
9,440.000000,0.014000


In [78]:
combine(groupby(iris, :Species), :SepalLength => (c -> first(c)) => "firstSpecies")

,Species,firstSpecies
,Cat…,Float64
1,setosa,5.100000
2,versicolor,7.000000
3,virginica,6.300000


In [79]:
select(groupby(iris, :Species), :SepalLength => (c -> minimum(c)))

,Species,SepalLength_function
,Cat…,Float64
1,setosa,4.300000
2,setosa,4.300000
3,setosa,4.300000
4,setosa,4.300000
5,setosa,4.300000
6,setosa,4.300000
7,setosa,4.300000
8,setosa,4.300000
9,setosa,4.300000


In [80]:
transform(groupby(iris, :Species), :SepalLength => (c -> first(c)))

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species,SepalLength_function
,Float64,Float64,Float64,Float64,Cat…,Float64
1,5.100000,3.500000,1.400000,0.200000,setosa,5.100000
2,4.900000,3.000000,1.400000,0.200000,setosa,5.100000
3,4.700000,3.200000,1.300000,0.200000,setosa,5.100000
4,4.600000,3.100000,1.500000,0.200000,setosa,5.100000
5,5.000000,3.600000,1.400000,0.200000,setosa,5.100000
6,5.400000,3.900000,1.700000,0.400000,setosa,5.100000
7,4.600000,3.400000,1.400000,0.300000,setosa,5.100000
8,5.000000,3.400000,1.500000,0.200000,setosa,5.100000
9,4.400000,2.900000,1.400000,0.200000,setosa,5.100000


## 嵌套list

In [2]:
using DataFrames
using CategoricalArrays

In [3]:
df = DataFrame(:x => [1, 2, 3, 4, 5, 1, 2, 3, 4, 5], :y => categorical(string.([:a, :a, :a, :b, :b, :b, :c, :c, :c, :c])))
first(df, 3)

,x,y
,Int64,Cat…
1,1,a
2,2,a
3,3,a


In [6]:
gdf  = groupby(df, :y);

In [7]:
for g in gdf
    println(g[:, :x])
end

[1, 2, 3]
[4, 5, 1]
[2, 3, 4, 5]


# 文件读写

In [12]:
using DelimitedFiles
using Random

## 写入矩阵

In [13]:
N = 100
x = rand(collect(1:10), N)
y = randn(N)
[x y];

In [14]:
pwd()

"C:\\Users\\xue\\Desktop\\Code\\笔记\\编程语言\\Julia"

In [15]:
writedlm("./Data/xy.txt", [x y], ',')

## 读取矩阵

In [16]:
mat = readdlm("./Data/xy.txt", ',');

## 创建CSV

In [17]:
using CSV
using DataFrames

In [18]:
df = DataFrame("x" => x, "y" => y)
first(df, 3)

,x,y
,Int64,Float64
1,1,0.135971
2,2,-0.227822
3,10,-0.212262


In [19]:
CSV.write("./Data/xy.csv", df)

"./Data/xy.csv"

## 读取CSV

In [20]:
df = CSV.read("./Data/xy.csv", DataFrame)
first(df, 3)

,x,y
,Int64,Float64
1,1,0.135971
2,2,-0.227822
3,10,-0.212262


In [21]:
value, header = readdlm("./Data/xy.csv", ','; header = true)

([1.000000 0.135971; 2.000000 -0.227822; … ; 8.000000 0.425795; 5.000000 -0.918026], AbstractString["x" "y"])